In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV

from sklearn.feature_selection import RFE
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

In [48]:
# model=Sequential()
# model.add(BatchNormalization())
# model.add(Dropout(0.20))
# model.add(Dense(2048,activation='relu',input_dim=train.shape[1],kernel_initializer='uniform'))
# model.add(BatchNormalization())
# model.add(Dense(2048,activation='tanh'))
# model.add(Dropout(0.2))
# model.add(Dense(1048,activation='tanh'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))
# #model.add(Dense(1024,kernel_initializer='uniform',activation='tanh'))
# #model.add(Dropout(0.5))
# model.add(Dense(target.shape[1],activation='sigmoid'))


# sgd=SGD(lr=0.01,momentum=0.9)
# model.compile(optimizer=sgd,loss='binary_crossentropy',metrics=['accuracy'])
# model.fit(train.values,target.values,batch_size=60,epochs=100,verbose=2)



In [49]:
df = pd.read_csv('data/serieA_matches.csv')
df_all_matches = pd.read_csv('data/2020_all_matches.csv')

In [50]:
%%time

home_teams = df_all_matches[['Home', 'Venue']].drop_duplicates().values
team_stadium = {}
for data in home_teams:
     team_stadium[data[0]] = data[1]

CPU times: user 4.17 ms, sys: 11 µs, total: 4.18 ms
Wall time: 4.71 ms


In [51]:
%%time

team_dict = {}
for idx, team in enumerate(df['team'].unique()):
    team_dict[team] = idx

CPU times: user 26.6 ms, sys: 0 ns, total: 26.6 ms
Wall time: 182 ms


In [52]:
df['is_home'] = team_stadium[df['team'].iloc[0]]
df['is_home'] = df['stadium'] == df['is_home']
df['is_home'] = df['is_home'].astype(int)

In [53]:
cols_types = df.dtypes
cols_types[cols_types == object]

team       object
result     object
stadium    object
state      object
UF         object
date       object
time       object
dtype: object

In [54]:
#

In [55]:
df.head(2)

,fouls,corners,crosses,touches,tackles,interceptions,aerials_won,clearances,offsides,goal_kicks,...,fouls_drawn_mean_attack,offsides_mean_attack,crosses_mean_attack,tackles_on_mean_attack,interceptions_mean_attack,own_goals_mean_attack,penalty_kicks_won_mean_attack,penalty_kicks_conceded_mean_attack,year,is_home
0,23.0,3.0,11.0,436.0,12.0,5.0,7.0,18.0,2.0,4.0,...,0.0,2.0,1.0,0.0,1.0,0.0,NaN,NaN,2019,0
1,23.0,3.0,11.0,436.0,12.0,5.0,7.0,18.0,2.0,4.0,...,0.0,2.0,1.0,0.0,1.0,0.0,NaN,NaN,2019,0


In [56]:
# df['date'] = pd.to_datetime(df['date'])
df['time'] = df['time'].apply(lambda time: time.split(':')[0]).astype(int)

In [57]:
# df['year'] = df['date'].dt.year
# df['month'] = df['date'].dt.month
# df['quarter'] = df['date'].dt.quarter
# df['day'] = df['date'].dt.day

In [58]:
cols_to_remove = [
    'result',
    "date",
#     'team',
    "stadium",
    "state",
    "UF",
#     "match_id"
]
y_col = 'score'

In [59]:
df.drop(columns=cols_to_remove, inplace=True)

In [60]:
feature_cols = [col for col in df.columns if col not in cols_to_remove]

In [61]:

df_home_teams = pd.DataFrame()
df_adversary_teams = pd.DataFrame()

df_teams = pd.DataFrame()
for team in df['team'].unique():
    cols_to_drop = ['is_home', 'team', 'score']

    df_team = df[df['team'] == team].copy().reset_index(drop=True)
    df_others_teams = df[
        (df['team'] != team) &
        (df['match_id'].isin(df_team['match_id'].tolist()))
    ].copy().reset_index(drop=True)
    
    columns = [col for col in df_team if col not in cols_to_drop]
    df_team.loc[df_team.index >= 3, columns] = (
        df_team[columns]
        .shift(1)
        .rolling(3)
        .mean()
        .iloc[3:]
    )

    df_others_teams.loc[df_others_teams.index >= 3, columns] = (
        df_team[columns]
        .shift(1)
        .rolling(3)
        .mean()
        .iloc[3:]
    )

    df_others_teams.drop(columns=cols_to_drop, inplace=True)
    df_others_teams.columns = [
        col + '_adversary' if col != 'match_id' else col for col in df_others_teams.columns
    ]
    
    for col in [column for column in df_others_teams.columns if column.endswith('adversary')]:
        df_team[col] = df_others_teams[col]
    
#     df_teams = df_teams.append(
#         df_team.merge(df_others_teams, on='match_id')
#     )


In [62]:
# df_teams = df_home_teams.append(
#     df_adversary_teams.merge(df_others_teams, on='match_id')
# )

df_teams = df_team

In [63]:
df_teams = df_teams[~df_teams['team'].isna()].copy()

In [64]:

df_teams['team'] = df_teams['team'].apply(lambda team: team_dict[team])

In [65]:
df_teams['score'].fillna(-1, inplace=True)
df_teams = df_teams[df_teams['score'] != -1]
df_teams['score'] = df_teams['score'].astype(int)

In [66]:
X = df_teams.drop(columns=['score'])
Y = df_teams['score']

In [67]:
# score
# result

# df.drop(columns=['result'], inplace=True)

In [68]:
# df.drop(columns=cols_to_remove, inplace=True)

In [69]:
# df

In [70]:
# model = XGBClassifier()

# rfe = RFE(model, 25)
# fit = rfe.fit(X, Y)
# print("Num Features: %d" % fit.n_features_)
# print("Selected Features: %s" % fit.support_)
# print("Feature Ranking: %s" % fit.ranking_)

In [71]:
# from sklearn.feature_selection import SelectKBest
# from sklearn.feature_selection import f_classif

# test = SelectKBest(score_func=f_classif, k=25)
# fit = test.fit(X, Y)
# # summarize scores
# print(fit.scores_)
# features = fit.transform(X)

In [72]:
# cols = np.array(np.array(X.columns.tolist()))[list(fit.support_)]

In [73]:
# X = X[cols]

In [74]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=7)

In [75]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    difference = list(np.abs((y_true - y_pred) / y_true))
    difference = [diff if diff != np.inf and not pd.isnull(diff) else 0 for diff in difference]
    return np.mean(difference)


In [76]:
def bias_percentage(y_true, y_pred): 
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    return  np.sum(y_pred) / np.sum(y_true)

In [77]:
X.sort_values(['matchweek', 'team'], inplace=True)

In [78]:
# # n_cols_list = [5, 15, 25, 35, 45, 50, 63, 72, 84, 90, 100]
# # for n_cols in n_cols_list:
n_cols = 50

# model = XGBRegressor()

# rfe = RFE(model, n_cols)
# fit = rfe.fit(X, Y)

# cols = np.array(np.array(X.columns.tolist()))[list(fit.support_)]
# X_new = X[cols]


In [79]:
X_new = X

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X_new, Y, test_size=0.1, random_state=7)

In [81]:
# %%time
# parameters = {
#     'eta': [0.05, 0.1, 0.3],
#     'max_depth': [5, 6, 9, 12, 20],
#     'subsample': [0.7, 0.6, 0.8, 1.0],
#     'colsample_bytree': [0.3, 0.6, 0.8, 1.0],
#     'min_child_weight': [1, 5, 8, 10],
#     'gamma': [0.5, 1, 1.5, 2, 5],
#     'n_estimators': [50, 100, 150, 500, 1000]
# }

# clf = GridSearchCV(model, parameters, n_jobs=1, cv=3)

# clf.fit(X_train, y_train)

# best_parameters = clf.best_params_

# model = XGBRegressor(**best_parameters)
model = XGBRegressor()

In [82]:
from sklearn.metrics import mean_absolute_error


model.fit(X_train, y_train)

y_pred = list(model.predict(X_test))
predictions = list([round(value) for value in y_pred])

# evaluate predictions
mape = mean_absolute_percentage_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
bias_percent = bias_percentage(y_test, predictions)

print("MAPE with {} columns: {:.2f}%".format(n_cols, mape * 100.0))
print("MAE with {} columns: {:.2f}".format(n_cols, mae))
print("BIAS% with {} columns: {:.2f}%".format(n_cols, bias_percent * 100))


MAPE with 50 columns: 0.60%
MAE with 50 columns: 0.01
BIAS% with 50 columns: 99.74%


<ipython-input-75-70d46a524b4a>:5: RuntimeWarning: divide by zero encountered in true_divide
  difference = list(np.abs((y_true - y_pred) / y_true))
<ipython-input-75-70d46a524b4a>:5: RuntimeWarning: invalid value encountered in true_divide
  difference = list(np.abs((y_true - y_pred) / y_true))


In [83]:
# param_test1 = {
#  'max_depth':range(3,10,2),
#  'min_child_weight':range(1,6,2)
# }
# gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
#  min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
#  objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
#  param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch1.fit(train[predictors],train[target])
# gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

# ==================================================

In [84]:
match_per_round = df['matchweek'].value_counts().to_frame()
last_matchweek = match_per_round[match_per_round > 5].sort_values('matchweek', ascending=False).iloc[0].name

In [85]:
next_matches = pd.read_csv('data/2020_all_matches.csv')

In [86]:
next_matches['Score'] = next_matches['Score'].fillna('')

In [87]:
next_matches = next_matches[next_matches['Score'] == '']
next_matches['Wk'] = next_matches['Wk'].fillna('')
next_matches = next_matches[next_matches['Wk'] != '']

In [88]:
next_matches = next_matches[next_matches['Wk'] > last_matchweek]

In [89]:
last_match_id = df['match_id'].max()

In [90]:
def make_dataset(interested_team, other_team, cols):
    cols_to_use = [col for col in interested_team.columns if 'adversary' not in col]
    
    interested_team = interested_team[cols_to_use]
    other_team = other_team[cols_to_use]
    
    other_team.columns = [col + '_adversary' if col not in ['result', 'match_id'] else col for col in interested_team.columns ]
    
    df = interested_team.merge(other_team, on='match_id')

    return df[cols]
    

In [107]:
for next_match in next_matches.iterrows():
    try:
        next_match = next_match[1].to_frame().transpose()
        team1, team2 = next_match[['Home', 'Away']].iloc[0]
        date = pd.to_datetime(next_match['Date'].iloc[0])
        if isinstance(next_match['Time'], str):
            time = next_match['Time'].iloc[0].split(':')[0]
        else:
            time = next_match['Time'].iloc[0]
            
        matchweek = int(next_match['Wk'].iloc[0])

        if team1 not in team_dict.keys():
            team_dict[team1] = max(team_dict.values()) + 1

        if team2 not in team_dict.keys():
            team_dict[team2] = max(team_dict.values()) + 1


        if matchweek >= last_matchweek + 1:

            cols = X_new.columns.tolist()
            mean_features_team1 = df[df['team'] == team1].copy()  # [cols]
            mean_features_team2 = df[df['team'] == team2].copy()  # [cols]

            mean_features_team1['match_id'] = last_match_id + 1
            mean_features_team2['match_id'] = last_match_id + 1

        #     for col in cols:
        #         mean_features_team1[col] = mean_features_team1[col].shift(1).rolling(window=3).mean().copy()
        #         mean_features_team2[col] = mean_features_team2[col].shift(1).rolling(window=3).mean().copy()

            for col in df.columns:
                if df[col].dtype != object and col not in ['score', 'match_id']:
                    mean_features_team1[col] = mean_features_team1[col].shift(1).rolling(window=3).mean().copy()
                    mean_features_team2[col] = mean_features_team2[col].shift(1).rolling(window=3).mean().copy()

            mean_features_team1 = mean_features_team1.iloc[-1:].copy()
            mean_features_team2 = mean_features_team2.iloc[-1:].copy()

            mean_features_team1['is_home'] = 1
            mean_features_team2['is_home'] = 0

            mean_features_team1['team'] = team_dict[team1]
            mean_features_team2['team'] = team_dict[team2]

            dataset_team1 = make_dataset(mean_features_team1, mean_features_team2, cols).round(2)
            dataset_team2 = make_dataset(mean_features_team2, mean_features_team1, cols).round(2)

            game_prediction = dataset_team1.append(dataset_team2)
            prediction = [round(prediction) for prediction in list(model.predict(game_prediction))]
            print(f"Rodada {matchweek}: {team1} {prediction[0]} x {prediction[1]} {team2}!")
            last_match_id += 1
            
            if matchweek == last_matchweek + 2:
                break
    except Exception as e:
#         print(e)
#         print(team1, team2)
        pass

Rodada 15: Vasco da Gama 1 x 2 Flamengo!
Rodada 15: Palmeiras 2 x 2 São Paulo!
Rodada 15: Atlético Mineiro 1 x 2 Goiás!
Rodada 15: Fluminense 1 x 1 Bahia!
Rodada 15: Santos 1 x 1 Grêmio!
Rodada 15: Ceará 2 x 1 Corinthians!
Rodada 15: Internacional 1 x 1 Atl Paranaense!
Rodada 11: Flamengo 1 x 2 Goiás!
